<a href="https://colab.research.google.com/github/sinha-diksha/Natural_Disaster_Prediction/blob/main/earthquake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [3]:
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
# Train-Test Split
from sklearn.model_selection import train_test_split
# Mean Squared Error (MSE)
from sklearn.metrics import mean_squared_error


In [43]:
import gdown


url = 'https://drive.google.com/file/d/1vQch-KWyGj1QdJWrdXaykn24Qycufxl4/view?usp=sharing' #earthquake.csv
file_id = url.split('/')[-2]
gdown_url = f'https://drive.google.com/uc?id={file_id}'
data = pd.read_csv(gdown_url)
print(data.head())

                                          title  magnitude         date_time  \
0  M 7.0 - 18 km SW of Malango, Solomon Islands        7.0  22-11-2022 02:03   
1      M 6.9 - 204 km SW of Bengkulu, Indonesia        6.9  18-11-2022 13:37   
2                                      M 7.0 -         7.0  12-11-2022 07:09   
3           M 7.3 - 205 km ESE of Neiafu, Tonga        7.3  11-11-2022 10:48   
4                                      M 6.6 -         6.6  09-11-2022 10:14   

   cdi  mmi  alert  tsunami  sig net  nst   dmin   gap magType    depth  \
0    8    7  green        1  768  us  117  0.509  17.0     mww   14.000   
1    4    4  green        0  735  us   99  2.229  34.0     mww   25.000   
2    3    3  green        1  755  us  147  3.125  18.0     mww  579.000   
3    5    5  green        1  833  us  149  1.865  21.0     mww   37.000   
4    0    2  green        1  670  us  131  4.998  27.0     mww  624.464   

   latitude  longitude                  location continent          

In [44]:
# Step 2: Feature Selection
selected_features = ['magnitude', 'cdi', 'mmi', 'tsunami', 'depth', 'latitude', 'longitude']


In [45]:
data

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
0,"M 7.0 - 18 km SW of Malango, Solomon Islands",7.0,22-11-2022 02:03,8,7,green,1,768,us,117,0.509,17.0,mww,14.000,-9.7963,159.596,"Malango, Solomon Islands",Oceania,Solomon Islands
1,"M 6.9 - 204 km SW of Bengkulu, Indonesia",6.9,18-11-2022 13:37,4,4,green,0,735,us,99,2.229,34.0,mww,25.000,-4.9559,100.738,"Bengkulu, Indonesia",NaN,NaN
2,M 7.0 -,7.0,12-11-2022 07:09,3,3,green,1,755,us,147,3.125,18.0,mww,579.000,-20.0508,-178.346,NaN,Oceania,Fiji
3,"M 7.3 - 205 km ESE of Neiafu, Tonga",7.3,11-11-2022 10:48,5,5,green,1,833,us,149,1.865,21.0,mww,37.000,-19.2918,-172.129,"Neiafu, Tonga",NaN,NaN
4,M 6.6 -,6.6,09-11-2022 10:14,0,2,green,1,670,us,131,4.998,27.0,mww,624.464,-25.5948,178.278,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,"M 7.7 - 28 km SSW of Puerto El Triunfo, El Sal...",7.7,13-01-2001 17:33,0,8,NaN,0,912,us,427,0.000,0.0,mwc,60.000,13.0490,-88.660,"Puerto El Triunfo, El Salvador",NaN,NaN
778,"M 6.9 - 47 km S of Old Harbor, Alaska",6.9,10-01-2001 16:02,5,7,NaN,0,745,ak,0,0.000,0.0,mw,36.400,56.7744,-153.281,"Old Harbor, Alaska",North America,NaN
779,"M 7.1 - 16 km NE of Port-Olry, Vanuatu",7.1,09-01-2001 16:49,0,7,NaN,0,776,us,372,0.000,0.0,mwb,103.000,-14.9280,167.170,"Port-Olry, Vanuatu",NaN,Vanuatu
780,"M 6.8 - Mindanao, Philippines",6.8,01-01-2001 08:54,0,5,NaN,0,711,us,64,0.000,0.0,mwc,33.000,6.6310,126.899,"Mindanao, Philippines",NaN,NaN


In [46]:
# Step 3: Data Cleaning
data = data.dropna(subset=selected_features)  # Remove rows with missing values in selected features


In [48]:
data.columns

Index(['title', 'magnitude', 'date_time', 'cdi', 'mmi', 'alert', 'tsunami',
       'sig', 'net', 'nst', 'dmin', 'gap', 'magType', 'depth', 'latitude',
       'longitude', 'location', 'continent', 'country'],
      dtype='object')

In [50]:
# Step 4: Data Transformation
non_numeric_columns = ['title', 'alert', 'magType', 'location', 'country', 'continent']
label_encoders = {}
for column in non_numeric_columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Convert 'magType' column into numeric labels
magType_encoder = LabelEncoder()
data['magType'] = magType_encoder.fit_transform(data['magType'])



#using Random Forest Classifier

In [51]:
# Step 5: Split the data into training and testing sets
X = data[selected_features]
y = data['magType']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [52]:
# Step 6: Model Selection and Training
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [53]:
# Step 7: Model Evaluation
y_train_pred = rf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

y_test_pred = rf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Train Accuracy:', train_accuracy)
print('Test Accuracy:', test_accuracy)


Train Accuracy: 1.0
Test Accuracy: 0.732484076433121


The code first uses the trained random forest model (rf) to predict the target variable for the training set (X_train). The accuracy score is then computed by comparing the predicted values (y_train_pred) with the actual target values (y_train). The same process is repeated for the test set (X_test, y_test). Finally, the train and test accuracy scores are printed.


In [56]:
data

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
0,504,7.0,22-11-2022 02:03,8,7,0,1,768,us,117,0.509,17.0,8,14.000,-9.7963,159.596,212,4,38
1,434,6.9,18-11-2022 13:37,4,4,0,0,735,us,99,2.229,34.0,8,25.000,-4.9559,100.738,48,6,49
2,487,7.0,12-11-2022 07:09,3,3,0,1,755,us,147,3.125,18.0,8,579.000,-20.0508,-178.346,413,4,13
3,619,7.3,11-11-2022 10:48,5,5,0,1,833,us,149,1.865,21.0,8,37.000,-19.2918,-172.129,253,6,49
4,127,6.6,09-11-2022 10:14,0,2,0,1,670,us,131,4.998,27.0,8,624.464,-25.5948,178.278,413,6,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,708,7.7,13-01-2001 17:33,0,8,4,0,912,us,427,0.000,0.0,7,60.000,13.0490,-88.660,307,6,49
778,458,6.9,10-01-2001 16:02,5,7,4,0,745,ak,0,0.000,0.0,5,36.400,56.7744,-153.281,270,3,49
779,547,7.1,09-01-2001 16:49,0,7,4,0,776,us,372,0.000,0.0,6,103.000,-14.9280,167.170,299,6,47
780,411,6.8,01-01-2001 08:54,0,5,4,0,711,us,64,0.000,0.0,7,33.000,6.6310,126.899,232,6,49


In [64]:
data[non_numeric_columns]

,title,alert,magType,location,country,continent
0,504,0,8,212,38,4
1,434,0,8,48,49,6
2,487,0,8,413,13,4
3,619,0,8,253,49,6
4,127,0,8,413,49,6
...,...,...,...,...,...,...
777,708,4,7,307,49,6
778,458,4,5,270,49,3
779,547,4,6,299,47,6
780,411,4,7,232,49,6


In [95]:
# Perform prediction on new data
new_data = pd.DataFrame({
    'title': ['M 7.0 - 18 km SW of Malango, Solomon Islands'],
    'magnitude': [7.0],
    'date_time': ['22/11/22 2:03'],
    'cdi': [8],
    'mmi': [7],
    'alert': ['green'],
    'tsunami': [1],
    'sig': [768],
    'net': ['us'],
    'nst': [117],
    'dmin': [0.509],
    'gap': [17],
    'depth': [14],
    'latitude': [-9.7963],
    'longitude': [159.596],
    'location': ['Malango, Solomon Islands'],
    'continent': ['Oceania'],
    'country': ['Solomon Islands']
})

new_data_encoded = new_data.copy()
print(new_data_encoded)

non_numeric_columns = ['title','alert', 'net', 'location', 'country', 'continent']
label_encoders = {}
for column in non_numeric_columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])






                                          title  magnitude      date_time  \
0  M 7.0 - 18 km SW of Malango, Solomon Islands        7.0  22/11/22 2:03   

   cdi  mmi  alert  tsunami  sig net  nst   dmin  gap  depth  latitude  \
0    8    7  green        1  768  us  117  0.509   17     14   -9.7963   

   longitude                  location continent          country  
0    159.596  Malango, Solomon Islands   Oceania  Solomon Islands  


In [97]:
# Ensure the column names match
# new_data_encoded = new_data_encoded[X.columns]

# Make prediction
prediction = rf.predict(new_data_encoded)
print('Predicted Magnitude Type:', prediction)

Predicted Magnitude Type: [8]


In [98]:
magType_counts = data['magType'].value_counts()
num_magTypes = len(magType_counts)

print("Number of different magTypes:", num_magTypes)
print("MagType value counts:")
print(magType_counts)

Number of different magTypes: 9
MagType value counts:
8    468
7    217
6     70
5     16
0      4
4      2
1      2
2      2
3      1
Name: magType, dtype: int64


printing the accuracy

In [99]:
print('Accuracy Data Train : ', rf.score(X_train, y_train))
print('Accuracy Data Test : ', rf.score(X_test, y_test))

Accuracy Data Train :  1.0
Accuracy Data Test :  0.732484076433121


Creating a confusion matrix

In [101]:

# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
confusion_matrix(y_test, y_test_pred)

array([[ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  1,  0,  0,  4],
       [ 0,  0,  0,  0,  3,  9,  4],
       [ 0,  0,  0,  0,  1, 27, 12],
       [ 0,  0,  0,  1,  0,  9, 83]])